In [0]:
!mkdir dataset
from google.colab import files
src = list(files.upload().values())[0]
open('\dataset','wb').write(src)
!mv sample_mobile_data_1000.csv dataset/
!ls dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install -q tensorflow-gpu==2.0.0a0
!pip install -q tensorboard
!pip install -q tensorboardcolab

In [2]:
from tensorboardcolab import *
tbc=TensorBoardColab(port=6006, graph_path='/tmp/logs', startup_waiting_time=8)

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://e0b1785e.ngrok.io
Wait for 8 seconds...
TensorBoard link:
http://e0b1785e.ngrok.io



Digit recognition :)



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import matplotlib.pyplot as plt
import os, datetime, time

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  # y labels are oh-encoded
# n_train = mnist.train.num_examples  # 55,000
# n_validation = mnist.validation.num_examples  # 5000
# n_test = mnist.test.num_examples  # 10,000

mnist = tf.keras.datasets.mnist
(x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = mnist.load_data()

In [0]:
def showtest(index):
  sample_digit = x_test_orig[index]
  sample_digit.reshape(28, 28)
  plt.imshow(sample_digit, cmap='gray')
  plt.show()
  print(y_test_orig[index])

In [0]:
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', 
                                     input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.sparse_categorical_crossentropy,
            metrics=['accuracy','mae', tf.keras.metrics.AUC(), tf.keras.metrics.SparseCategoricalAccuracy()])
    return model
 
# Create a basic model instance
model = create_model()

In [0]:
# model.summary()

In [126]:
w, h = 28, 28
x_train = x_train_orig.reshape(x_train.shape[0], w, h, 1)
x_test = x_test_orig.reshape(x_test.shape[0], w, h, 1)
logdir="/tmp/logs/1"
checkpoint_path = logdir + "/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=checkpoint_path)

fit = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10,
                callbacks = [cp_callback, tb_callback], use_multiprocessing=True, batch_size=100,
               validation_split=0.8)

ans = model.evaluate(x_test, y_test)

Train on 60000 samples, validate on 10000 samples


InvalidArgumentError: ignored

In [0]:
wrong=[]
for ind in range(1000):
  res = model.evaluate(x_test[0:ind + 1], y_test[0:ind + 1], verbose=0)
  if res[1] < 0.5:
    wrong.append(ind)

In [82]:
inp = wrong[1]
showtest(inp)
model.predict(x_test[inp:inp+1])

IndexError: ignored

In [0]:
!ls -lah {checkpoint_dir}
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(x_test, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

total 1.7M
drwxr-xr-x 2 root root 4.0K May 22 02:36 .
drwxr-xr-x 1 root root 4.0K May 22 02:33 ..
-rw-r--r-- 1 root root   71 May 22 02:36 checkpoint
-rw-r--r-- 1 root root 1.6M May 22 02:36 cp.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root 1.3K May 22 02:36 cp.ckpt.index
10000/10000 [==============================] - 1s 66us/sample - loss: 0.0404 - acc: 0.9884
Restored model, accuracy: 98.84%
